*Internal Note:* 

- *The following is a tutorial template to be used by PO.DAAC staff when developing end-user tutorials.*
- *Copy this template and edit to create your notebook, keeping as many key elements as appropriate.*
- *DO NOT modify this template to create a tutorial. If you wish to make suggestions to improve the template please submit a PR and tag Catalina Oaida Taglialatela (ScienceCat18)*

> From the PO.DAAC Cookbook, to access the GitHub version of the notebook, follow [this link](insert link to notebook).

# SWOT Hydro Science Application Tutorial
#### *Authors: Cerbelaud A., Wade J., Jet Propulsion Laboratory - Caltech*

## Summary

_[Add summary/tutorial description here]_

## Requirements

### 1. Compute environment 
*internal note (remove this note in final tutorial): keep one or both of these Req 1 depending on environment required to run the noteook*

This tutorial can be run in the following environments:
- **AWS instance running in us-west-2**: NASA Earthdata Cloud data in S3 can be directly accessed via temporary credentials; this access is limited to requests made within the US West (Oregon) (code: `us-west-2`) AWS region.
- **Local compute environment** e.g. laptop, server: this tutorial can be run on your local machine

### 2. Earthdata Login

An Earthdata Login account is required to access data, as well as discover restricted data, from the NASA Earthdata system. Thus, to access NASA data, you need Earthdata Login. Please visit https://urs.earthdata.nasa.gov to register and manage your Earthdata Login account. This account is free to create and only takes a moment to set up.

### 3. Additional Requirements

Any other requirements needed for reproducing this tutorial.

## Learning Objectives
- enter objective 
- enter objective 
- ...

------

## Import Packages

In [1]:
import glob
import os
import requests
import s3fs
import fiona
import netCDF4 as nc
import h5netcdf
import xarray as xr
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import hvplot.xarray
import earthaccess
from earthaccess import Auth, DataCollections, DataGranules, Store

## Authenticate
Authenticate your Earthdata Login (EDL) information using the `earthaccess` python package as follows:

In [2]:
auth = earthaccess.login() # Login with your EDL credentials if asked

EARTHDATA_USERNAME and EARTHDATA_PASSWORD are not set in the current environment, try setting them or use a different strategy (netrc, interactive)
No .netrc found in /home/jovyan


Enter your Earthdata Login username:  jswade
Enter your Earthdata password:  ········


You're now authenticated with NASA Earthdata Login
Using token with expiration date: 03/16/2024
Using user provided credentials for EDL


#### Search for the data of interest

In [3]:
#Retrieves granule from the day we want, in this case by passing to `earthdata.search_data` function the data collection shortname, temporal bounds, and for restricted data one must specify the search count
river_results = earthaccess.search_data(short_name = 'SWOT_L2_HR_RIVERSP_1.1', 
                                        temporal = ('2023-04-08 00:00:00', '2023-04-22 23:59:59'),
                                        granule_name = '*Reach*_013_NA*') # here we filter by Reach files (not node), pass #13 and continent code=NA

Granules found: 14


#### Set up an `s3fs` session for Direct Cloud Access
`s3fs` sessions are used for authenticated access to s3 bucket and allows for typical file-system style operations. Below we create session by passing in the data access information.

In [4]:
fs_s3 = earthaccess.get_s3fs_session(results=river_results)

#### Create Fiona session to work with zip and embedded shapefiles in the AWS Cloud
The native format for this data is a **.zip** file, and we want the **.shp** file within the .zip file, so we will create a Fiona AWS session using the credentials from setting up the s3fs session above to access the shapefiles within the zip files. If we don't do this, the alternative would be to download the data to the cloud environment (e.g. EC2 instance, user S3 bucket) and extract the .zip file there.

In [5]:
fiona_session=fiona.session.AWSSession(
        aws_access_key_id=fs_s3.storage_options["key"],
        aws_secret_access_key=fs_s3.storage_options["secret"],
        aws_session_token=fs_s3.storage_options["token"]
    )

In [18]:
# Initialize list to shp files
SWOT_HR_shps = []

# Loop through queried granules
for j in range(0,len(river_results)):
    
    # Get the link for each zip file
    river_link = earthaccess.results.DataGranule.data_links(river_results[j], access='direct')[0]
    
    # We use the zip+ prefix so fiona knows that we are operating on a zip file
    river_shp_url = f"zip+{river_link}"
    
    # Read shapefile
    with fiona.Env(session=fiona_session):
        SWOT_HR_shps.append(gpd.read_file(river_shp_url)) 



In [26]:
SWOT_HR_df = gpd.GeoDataFrame(pd.concat(SWOT_HR_shps, ignore_index=True))

,reach_id,time,time_tai,time_str,p_lat,p_lon,river_name,wse,wse_u,wse_r_u,...,p_wid_var,p_n_nodes,p_dist_out,p_length,p_maf,p_dam_id,p_n_ch_max,p_n_ch_mod,p_low_slp,geometry
0,77460000013,7.342535e+08,7.342536e+08,2023-04-08T07:18:42Z,40.621824,-124.244823,Eel River,2.320100e+00,9.653000e-02,3.490000e-02,...,23702.805,96,19376.986,19194.609276,-1.000000e+12,0,4,1,0,"LINESTRING (-124.29069 40.66364, -124.29104 40..."
1,77460000021,7.342535e+08,7.342536e+08,2023-04-08T07:18:42Z,40.542406,-124.156177,Eel River,9.248800e+00,9.071000e-02,1.130000e-02,...,3435.524,51,29478.836,10101.849934,-1.000000e+12,0,2,1,0,"LINESTRING (-124.16119 40.58421, -124.16097 40..."
2,77460000031,7.342535e+08,7.342536e+08,2023-04-08T07:18:42Z,40.494638,-124.107178,Eel River,1.970160e+01,5.754600e-01,5.683800e-01,...,1202.549,50,39553.707,10074.871060,-1.000000e+12,0,2,1,0,"LINESTRING (-124.13864 40.50871, -124.13829 40..."
3,77460000041,7.342535e+08,7.342536e+08,2023-04-08T07:18:42Z,40.447111,-124.021272,Eel River,3.471650e+01,1.486718e+01,1.486691e+01,...,645.984,81,55843.425,16289.718636,-1.000000e+12,0,1,1,0,"LINESTRING (-124.09611 40.46269, -124.09575 40..."
4,77460000051,7.342535e+08,7.342536e+08,2023-04-08T07:18:42Z,40.395990,-123.930243,Eel River,3.229870e+01,9.148000e-02,1.638000e-02,...,1532.658,51,66073.910,10230.484650,-1.000000e+12,0,2,1,0,"LINESTRING (-123.95755 40.42295, -123.95719 40..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3871,78322900143,-1.000000e+12,-1.000000e+12,no_data,50.998910,-119.011806,no_data,-1.000000e+12,-1.000000e+12,-1.000000e+12,...,116183.841,15,59585.535,2935.007230,-1.000000e+12,0,1,1,0,"LINESTRING (-119.03134 51.00406, -119.03105 51..."
3872,78322900153,7.354554e+08,7.354555e+08,2023-04-22T05:10:43Z,51.047442,-119.042215,no_data,3.464306e+02,7.581200e-01,7.527600e-01,...,41059.043,51,30434.699,10168.113068,-1.000000e+12,0,2,1,0,"LINESTRING (-119.03140 51.09147, -119.03165 51..."
3873,78322900173,-1.000000e+12,-1.000000e+12,no_data,50.916115,-119.036919,no_data,-1.000000e+12,-1.000000e+12,-1.000000e+12,...,186564.361,98,39488.857,19642.879384,-1.000000e+12,0,4,1,0,"LINESTRING (-118.99201 50.99421, -118.99200 50..."
3874,78322900183,-1.000000e+12,-1.000000e+12,no_data,51.060235,-118.938606,no_data,-1.000000e+12,-1.000000e+12,-1.000000e+12,...,177029.325,86,56650.527,17161.670624,-1.000000e+12,0,4,1,0,"LINESTRING (-118.99200 50.99394, -118.99201 50..."
